# Using the NeuroRobotics Platform with SpiNNaker
This will guide you in using the HBP NRP through the SpiNNaker Jupyter notebooks.  An example of the execution of a model that has been set up to run with SpiNNaker will be given.

## Starting the NRP from Jupyter
Once you are logged in to Jupyter, Start a terminal as shown in the diagram below.
<img src="JupyterTerminal.png"/>

Once the terminal has loaded, you can start the NRP by executing the commands:
<pre>
   cle-nginx
   cle-start
</pre>
<img src="JupyterStartNRP.png">

One the NRP has started, you can now access your own NRP on a URL like the following, replacing ```<username>``` with your username:

    https://spinn-20.cs.man.ac.uk/user/<username>/proxy/9000/#/esv-private

This will result in a login screen like the following:
<img src="NRPLogin.png">

Use the username ```nrpuser``` and the password ```password``` to log in.

## Using the NRP to run an Experiment on SpiNNaker
When you are logged in, you will see a "Templates" screen.  Look for an experiment called "Holodeck Husky Braitenberg Example on SpiNNaker".  Select this, and select the "Clone" button.
<img src="NRPTemplates.png">

You will now see the experiment in the "My experiments" tab.  Select the cloned example, and click on "Launch" to start the environment.
<img src="NRPMyExperiments.png">

You will now see a loading screen:
<img src="NRPLoading.png">

When loading is complete, you will see the NRP environment.  Click on the Play button to start the execution.
<img src="NRPPlay.png">

The execution on SpiNNaker will now happen.  This will take a little while to happen; if you want, you can return to your Terminal tab to see the execution process.
<img src="NRPPlaying.png">

Once the simulation starts, the Robot should start spinning on the spot.  You can now interact with the simulation.  The robot is looking for a red screen, so you can right-click on one of the screens to turn it red.
<img src="NRPScreens.png">

Once the robot moves toward the screen, you can turn it blue again.  If you then turn the opposite screen red, the robot will turn until it sees that and move towards it.

Once you have finished the simulation, select the Exit button to shut down the simulation.
<img src="NRPStop.png">

You can then stop the NRP itself from the Terminal again, by typing the CTRL-C keys and then typing ```cle-kill```.
<img src="JupyterNRPKill.png">

## NRP Files

Inside the ```nrpStorage``` folder there should now be a sub-folder called ```braitenberg_husky_holodeck_spinnaker_0```.  This contains all the experiment files.  In particular there are the following files:

 - braitenberg_spinnaker.py - This is the brain of the Robot as a PyNN script that will run on SpiNNaker.
 - eye_sensor_transmit.py - This is a Transfer Function that reads the eye sensors of the robot and then sends messages to SpiNNaker as the input to the brain.
 - linear_twist.py - This is a Transfer Function that reads the membrane voltage of neurons running on SpiNNaker and controls the speed of the robot motors.
 - bibi_configuration.bibi - This is the configuration file that joins together the parts of the experiment, as well as indicating that it should run on SpiNNaker.

### The Brain PyNN Description
The Brain description is a neural network that runs on SpiNNaker.  Note that this only contains the neurons of the brain, not the input and output devices.  These devices are added by the Transfer Functions (see later).

The Brain network used by the example is shown below:

In [ ]:
from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)


def create_brain():
    """
    Initializes PyNN with the neuronal network that has to be simulated
    """
    SENSORPARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.0,
                    'v_reset': -60.5}

    GO_ON_PARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_reset': -61.6,
                    'v_thresh': -60.51,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5}

    # POPULATION_PARAMS = SENSORPARAMS * 5 + GO_ON_PARAMS + SENSORPARAMS * 2
    red_left_eye = sim.Population(2, sim.IF_cond_exp(**SENSORPARAMS), label="red_left_eye")
    red_right_eye = sim.Population(2, sim.IF_cond_exp(**SENSORPARAMS), label="red_right_eye")
    green_blue_eye = sim.Population(1, sim.IF_cond_exp(**SENSORPARAMS), label="green_blue_eye")
    go_on = sim.Population(1, sim.IF_cond_exp(**GO_ON_PARAMS), label="go_on")
    left_wheel_motor = sim.Population(1, sim.IF_cond_exp(**SENSORPARAMS), label="left_wheel_motor")
    right_wheel_motor = sim.Population(1, sim.IF_cond_exp(**SENSORPARAMS), label="right_wheel_motor")

    # population = sim.Population(8, sim.IF_cond_exp())
    # population[0:5].set(**SENSORPARAMS) # 0, 2 = red_left_eye, 1, 3 = red_right_eye, 4 = green_blue_eye
    # population[5:6].set(**GO_ON_PARAMS) # 5 = go_on?
    # population[6:8].set(**SENSORPARAMS) # 6 = left_wheel_motor, 7=right_wheel_motor

    # Shared Synapse Parameters
    # syn_params = {'U': 1.0, 'tau_rec': 1.0, 'tau_facil': 1.0}

    # Synaptic weights
    WEIGHT_RED_TO_ACTOR = 1.5e-4
    WEIGHT_RED_TO_GO_ON = 1.2e-3  # or -1.2e-3?
    WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-4
    WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4
    DELAY = 0.1

    # Connect neurons

    SYN = sim.StaticSynapse(weight=abs(WEIGHT_RED_TO_ACTOR), delay=DELAY)
    sim.Projection(red_left_eye, # red_left_eye[1]
                   right_wheel_motor, # right_wheel_motor
                   connector=sim.FromListConnector([(1, 0)]),
                   synapse_type=SYN,
                   receptor_type='excitatory')
    sim.Projection(red_right_eye, # red_right_eye[1]
                   left_wheel_motor, # left_wheel_motor
                   connector=sim.FromListConnector([(1, 0)]),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    SYN = sim.StaticSynapse(weight=abs(WEIGHT_RED_TO_GO_ON), delay=DELAY)
    sim.Projection(red_left_eye, # red_left_eye
                   green_blue_eye, # green_blue_eye
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')
    sim.Projection(red_left_eye, # red_left_eye
                   go_on, # go_on
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')

    SYN = sim.StaticSynapse(weight=abs(WEIGHT_GREEN_BLUE_TO_ACTOR), delay=DELAY)
    sim.Projection(green_blue_eye, # green_blue_eye
                   right_wheel_motor, # right_wheel_motor
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    SYN = sim.StaticSynapse(weight=abs(WEIGHT_GO_ON_TO_RIGHT_ACTOR), delay=DELAY)
    sim.Projection(go_on, # go_on
                   right_wheel_motor, # right_wheel_motor
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    # sim.initialize(population, v=population.get('v_rest'))

    # logger.debug("Circuit description: " + str(population.describe()))

    return {"red_left_eye": red_left_eye, "red_right_eye": red_right_eye,
            "green_blue_eye": green_blue_eye, "go_on": go_on,
            "left_wheel_motor": left_wheel_motor, "right_wheel_motor": right_wheel_motor}


circuit = create_brain()


This creates several ```Population``` objects and connects them together using ```Projection``` as normal in a PyNN network.  The only difference is that at the end of the script, a dictionary is created called ```circuit``` which contains the Populations that will be used in the NRP experiment.

### Transfer Functions
Once the Brain has been defined, Transfer Functions can be written which will communicate between the Brain and the Robot.  The Transfer Function that sends the input into the network from the eyes of the Robot is shown below:

In [ ]:
@nrp.MapRobotSubscriber("camera", Topic('/husky/husky/camera', sensor_msgs.msg.Image))
@nrp.MapSpikeSource("red_left_eye", nrp.brain.red_left_eye, nrp.poisson)
@nrp.MapSpikeSource("red_right_eye", nrp.brain.red_right_eye, nrp.poisson)
@nrp.MapSpikeSource("green_blue_eye", nrp.brain.green_blue_eye, nrp.poisson)
@nrp.Robot2Neuron()
def eye_sensor_transmit(t, camera, red_left_eye, red_right_eye, green_blue_eye):
    image_results = hbp_nrp_cle.tf_framework.tf_lib.detect_red(image=camera.value)
    red_left_eye.rate = 2000.0 * image_results.left
    red_right_eye.rate = 2000.0 * image_results.right
    green_blue_eye.rate = 75.0 * image_results.go_on


This first sets up a link from the robot camera to the function argument "camera".  This uses ROS to communicate with the camera.

Next, three of the ```Population``` objects are connected to externally controlled Poisson sources.  Internally, this creates a Poisson source for each of the Populations, and then creates a Projection from each to the Population respectively.  The Poisson sources are each provided as arguments to the function.

Finally, the function is marked as ```Robot2Neuron``` indicating it is inputting into the Brain.

The function itself takes the camera image and performs some processing on it to extract which parts are red and which are green and blue.  The function takes these results and updates the Poisson rates of the network.

A Transfer Function that controls the motors is shown below:

In [ ]:
@nrp.MapSpikeSink("left_wheel_neuron", nrp.brain.left_wheel_motor, nrp.leaky_integrator_exp, weight=0.1)
@nrp.MapSpikeSink("right_wheel_neuron", nrp.brain.right_wheel_motor, nrp.leaky_integrator_exp, weight=0.1)
@nrp.Neuron2Robot(Topic('/husky/husky/cmd_vel', geometry_msgs.msg.Twist))
def linear_twist(t, left_wheel_neuron, right_wheel_neuron):
    return geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=20.0 * min(left_wheel_neuron.voltage, right_wheel_neuron.voltage), y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=100.0 * (right_wheel_neuron.voltage - left_wheel_neuron.voltage)))


This first sets up Populations from which the Membrane voltage can be read.  Like the Poisson sources in the previous example, these are in addition to the network described in the brain.  These are connected to the specified brain Populations using a Projection; in this case the weight is explicitly specified.  These are then provided to the function as the arguments specified.

The next line tells the platform that the robot will be sent a ```Twist``` object through the ROS topic ```/husky/husky/cmd_vel```.  The function then takes the voltages of the neurons and returns this object.

### Bibi File
The final part of the experiment is the bibi file.  This connects the parts of the experiment together.  Below is an example:

In [ ]:
<ns1:bibi 
  xmlns:ns1="http://schemas.humanbrainproject.eu/SP10/2014/BIBI" 
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <ns1:brainModel>
    <ns1:file>braitenberg_spinnaker.py</ns1:file>
    <ns1:populations population="red_left_eye" count="2" xsi:type="ns1:Population" />
    <ns1:populations population="red_right_eye" count="2" xsi:type="ns1:Population" />
    <ns1:populations population="green_blue_eye" count="1" xsi:type="ns1:Population" />
    <ns1:populations population="go_on" count="1" xsi:type="ns1:Population" />
    <ns1:populations population="left_wheel_motor" count="1" xsi:type="ns1:Population" />
    <ns1:populations population="right_wheel_motor" count="1" xsi:type="ns1:Population" />
  </ns1:brainModel>
  <ns1:bodyModel robotId="husky">husky_model/model.sdf</ns1:bodyModel>
  <ns1:mode>SynchronousSpinnakerSimulation</ns1:mode>
  <ns1:transferFunction src="csv_spike_monitor.py" xsi:type="ns1:PythonTransferFunction" />
  <ns1:transferFunction src="csv_joint_state_monitor.py" xsi:type="ns1:PythonTransferFunction" />
  <ns1:transferFunction src="csv_robot_position.py" xsi:type="ns1:PythonTransferFunction" />
  <ns1:transferFunction src="all_neurons_spike_monitor.py" xsi:type="ns1:PythonTransferFunction" />
  <ns1:transferFunction src="linear_twist.py" xsi:type="ns1:PythonTransferFunction" />
  <ns1:transferFunction src="eye_sensor_transmit.py" xsi:type="ns1:PythonTransferFunction" />
</ns1:bibi>

Here, the Populations of the Brain model are specified.  The body of the robot is then linked.  Following this, the system is told that the network is to be simulated using SpiNNaker with the mode ```SynchronousSpinnakerSimulation```.  The transfer functions are then listed.